## データベースの構造

### `pagelinks`

| Field               | Type                | Null | Key | Default | Extra |
| ------------------- | ------------------- | ---- | --- | ------- | ----- |
| `pl_from`           | int(10) unsigned    | NO   | PRI | 0       |       |
| `pl_target_id`      | bigint(20) unsigned | NO   | PRI | NULL    |       |
| `pl_from_namespace` | int(11)             | NO   | MUL | 0       |       |

### `linktarget`

| Field          | Type                | Null | Key | Default | Extra          |
| -------------- | ------------------- | ---- | --- | ------- | -------------- |
| `lt_id`        | bigint(20) unsigned | NO   | PRI | NULL    | auto_increment |
| `lt_namespace` | int(11)             | NO   | MUL | NULL    |                |
| `lt_title`     | varbinary(255)      | NO   |     | NULL    |                |

### `page`

| Field                | Type                | Null | Key | Default | Extra          |
| -------------------- | ------------------- | ---- | --- | ------- | -------------- |
| `page_id`            | int(10) unsigned    | NO   | PRI | NULL    | auto_increment |
| `page_namespace`     | int(11)             | NO   | MUL | NULL    |                |
| `page_title`         | varbinary(255)      | NO   |     | NULL    |                |
| `page_is_redirect`   | tinyint(3) unsigned | NO   | MUL | 0       |                |
| `page_is_new`        | tinyint(3) unsigned | NO   |     | 0       |                |
| `page_random`        | double unsigned     | NO   | MUL | NULL    |                |
| `page_touched`       | binary(14)          | NO   |     | NULL    |                |
| `page_links_updated` | binary(14)          | YES  |     | NULL    |                |
| `page_latest`        | int(10) unsigned    | NO   |     | NULL    |                |
| `page_len`           | int(10) unsigned    | NO   | MUL | NULL    |                |
| `page_content_model` | varbinary(32)       | YES  |     | NULL    |                |
| `page_lang`          | varbinary(35)       | YES  |     | NULL    |                |

### `redirect`

| Field          | Type             | Null | Key | Default | Extra |
| -------------- | ---------------- | ---- | --- | ------- | ----- |
| `rd_from`      | int(10) unsigned | NO   | PRI | 0       |       |
| `rd_namespace` | int(11)          | NO   | MUL | 0       |       |
| `rd_title`     | varbinary(255)   | NO   |     | NULL    |       |
| `rd_interwiki` | varbinary(32)    | YES  |     | NULL    |       |
| `rd_fragment`  | varbinary(255)   | YES  |     | NULL    |       |


In [2]:
from pathlib import Path
import sqlite3


wikidata = Path("../wikidata")
page_db = wikidata / "jawiki-latest-page.sqlite"
pagelink_db = wikidata / "jawiki-latest-pagelinks.sqlite"
redirect_db = wikidata / "jawiki-latest-redirect.sqlite"
linktarget_db = wikidata / "jawiki-latest-linktarget.sqlite"
gen_db = wikidata / "gen.sqlite"

In [3]:
# memory上にデータベースを作成して、それぞれのデータベースをアタッチする
conn = sqlite3.connect(f"{gen_db}")
cur = conn.cursor()
_ = cur.execute("ATTACH DATABASE ? AS pagelinks", (str(pagelink_db),))
_ = cur.execute("ATTACH DATABASE ? AS redirect", (str(redirect_db),))
_ = cur.execute("ATTACH DATABASE ? AS page", (str(page_db),))
_ = cur.execute("ATTACH DATABASE ? AS linktarget", (str(linktarget_db),))

In [4]:
# テーブルをコピーする  namespaceが0(記事)かつ日本語wikiのものだけを対象とする
_ = cur.execute("CREATE TABLE page AS SELECT * FROM page.page WHERE page_namespace = 0")
_ = cur.execute("CREATE TABLE pagelinks AS SELECT * FROM pagelinks.pagelinks WHERE pl_from_namespace = 0")
_ = cur.execute("CREATE TABLE redirect AS SELECT * FROM redirect.redirect WHERE rd_namespace = 0 AND rd_interwiki = ''")
_ = cur.execute("CREATE TABLE linktarget AS SELECT * FROM linktarget.linktarget WHERE lt_namespace = 0")

In [5]:
# indexを作成
_ = cur.execute("CREATE INDEX IF NOT EXISTS idx_page_title ON page (page_title)")
_ = cur.execute("CREATE INDEX IF NOT EXISTS idx_rd_title ON redirect (rd_title)")
_ = cur.execute("CREATE INDEX IF NOT EXISTS idx_lt_title ON linktarget (lt_title)")

### redirect 先の page が存在しているものだけを取得して，リダイレクト先の id を結合

- `rd_namespace = 0 AND rd_interwiki = ''` で，リダイレクト先が日本語 wiki の記事のものだけを取得しているが，リダイレクト元がそうであるとは限らない．リダイレクト元が記事じゃないページが混じっている．
  [例](https://ja.wikipedia.org/w/index.php?title=%E5%88%A9%E7%94%A8%E8%80%85:%E6%97%A5%E6%9C%AC%E6%B5%B7%EF%BC%90%EF%BC%93%E4%B8%B8/sandbox&redirect=no)
- redirect 先が存在しないものもある [例](https://ja.wikipedia.org/w/index.php?curid=1690433)
- 上は `INNER JOIN` で消える


In [6]:
sql = """
CREATE TABLE redirect_id AS
SELECT
    rd_from AS rd_from_id,
    page.page_id AS rd_to_id
FROM
    redirect
INNER JOIN
    page
ON
    rd_title = page.page_title
;
"""
_ = cur.execute(sql)

### redirect 先を展開して，リンク元とリンク先の id を結合

- `redirect_id` のリダイレクト元が記事でないやつが INNER JOIN で消える
- リダイレクト先が記事でないやつもあり， `page_redirect_id IS NOT NULL` で消える [例](https://ja.wikipedia.org/w/index.php?curid=884763)


In [ ]:
sql = """
CREATE TABLE page_and_redirect AS
SELECT
    page.page_id,
    page.page_title,
    page.page_is_redirect,
    rd_to_id as page_redirect_id
FROM
    page
LEFT JOIN
    redirect_id
ON
    page.page_id = redirect_id.rd_from_id
WHERE
    page.page_is_redirect =0 
OR
    page.page_is_redirect =1 AND page_redirect_id IS NOT NULL
;
"""
_ = cur.execute(sql)
_ = cur.execute("CREATE INDEX IF NOT EXISTS idx_page_id ON page_and_redirect (page_id)")

### linktarget 先の page が記事のものだけを取得して，リンク先の id を結合

- そもそも linktarge 先が存在しないものもあるのでそういうのも INNER JOIN で消える
- [このページ](https://ja.wikipedia.org/wiki/%E5%88%A9%E7%94%A8%E8%80%85:D%C9%90/List_of_all_single-letter-double-digit_combinations)
  のような存在しないリンク先を保持したページが多々あり，リンク先が存在しないものがめっちゃある


In [8]:
sql = """
CREATE TABLE linktarget_id AS
SELECT
    lt_id AS lt_from_id,
    page_and_redirect.page_id AS lt_to_id
FROM
    linktarget
INNER JOIN
    page_and_redirect
ON
    linktarget.lt_title = page_and_redirect.page_title
;
"""
_ = cur.execute(sql)

### pagelinks の先が page が記事のものだけを取得して，リンク先の id を結合

- `pagelinks`ではすでに`pl_from`が記事のものだけを取得しているが，リンク先が記事でないものもある
- `lintarget`の方は記事以外のものは存在しないので，`pagelinks`にはあるが，`linktarget`にはないものがあるので，INNER JOIN で消える

### pagelinks のリンク先とリンク元が page に存在するものだけを取得

- INNER JOIN でないものは消える


In [9]:
sql = """
CREATE TABLE pagelinks_id AS
SELECT
    pl_from AS pl_from_id,
    linktarget_id.lt_to_id AS pl_to_id
FROM
    pagelinks
INNER JOIN
    linktarget_id
ON
    pagelinks.pl_target_id = linktarget_id.lt_from_id
INNER JOIN
    page_and_redirect par_from
ON
    pagelinks.pl_from = par_from.page_id
INNER JOIN
    page_and_redirect par_to
ON
    linktarget_id.lt_to_id = par_to.page_id
;
"""
_ = cur.execute(sql)
_ = cur.execute("CREATE INDEX IF NOT EXISTS idx_pl_to_id ON pagelinks_id (pl_to_id)")
_ = cur.execute("CREATE INDEX IF NOT EXISTS idx_pl_from_id ON pagelinks_id (pl_from_id)")

### pagelinks_id と page_and_redirect 以外のテーブルを削除


In [10]:
# pagelinks_idとpage_and_redirect以外のテーブルを削除
_ = cur.execute("DROP TABLE IF EXISTS page")
_ = cur.execute("DROP TABLE IF EXISTS pagelinks")
_ = cur.execute("DROP TABLE IF EXISTS redirect")
_ = cur.execute("DROP TABLE IF EXISTS linktarget")
_ = cur.execute("DROP TABLE IF EXISTS redirect_id")
_ = cur.execute("DROP TABLE IF EXISTS linktarget_id")